# Test `scan` and `scan_layers` in Llama 3

Hugging Face usage follows https://github.com/huggingface/notebooks/blob/main/examples/language_modeling.ipynb

To test scan, we need to use a custom modification of the transformer repo:
https://github.com/tengyifei/transformers/tree/test-scan

In [1]:
%env PJRT_DEVICE=TPU
%env XLA_USE_SPMD=1

env: PJRT_DEVICE=TPU
env: XLA_USE_SPMD=1


In [2]:
import torch
import torch_xla
import numpy as np
import transformers

/workspaces/torch/pytorch/xla/torch_xla/__init__.py:250: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def reset_seed():
  torch.random.manual_seed(42)
  torch_xla.manual_seed(42)
  np.random.seed(42)

In [4]:
from datasets import load_dataset

dataset = load_dataset("Salesforce/wikitext", "wikitext-103-v1")

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
tokenizer.bos_token_id = 128000
tokenizer.eos_token_id = 128001
tokenizer.pad_token_id = tokenizer.eos_token_id 

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"], batch_size=1000)

In [6]:
tokenized_datasets.keys()  # type:ignore

dict_keys(['test', 'train', 'validation'])

In [7]:
tokenized_datasets["train"][1].keys()  # type:ignore

dict_keys(['input_ids', 'attention_mask'])

In [8]:
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
)

In [9]:
lm_datasets["train"][1].keys(), lm_datasets["validation"][1].keys()  # type:ignore

(dict_keys(['input_ids', 'attention_mask', 'labels']),
 dict_keys(['input_ids', 'attention_mask', 'labels']))

In [10]:
len(lm_datasets["validation"])  # type:ignore

3760

In [11]:
from transformers import LlamaConfig, LlamaForCausalLM

reset_seed()

# Define model configuration
config = LlamaConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=1024,  # Model size
    num_hidden_layers=28,  # Number of transformer layers
    num_attention_heads=8,  # Number of attention heads
    intermediate_size=2048,  # Size of the hidden feedforward layer
    max_position_embeddings=128,  # Max tokens in a sequence
    use_cache=False,
    unroll_decoders=True,
)

# Instantiate the model
model = LlamaForCausalLM(config)

2024-11-21 01:58:04.865781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732154284.875010 3460421 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732154284.877923 3460421 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=1,
    max_steps=4000,
    save_strategy="no",
    save_total_limit=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=500,
    gradient_accumulation_steps=1,
    fp16=False,
    bf16=False,
    tpu_num_cores=4,
    push_to_hub=False,
)

In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"].shuffle(seed=42),  # type:ignore
    eval_dataset=lm_datasets["validation"].shuffle(seed=42),  # type:ignore
    processing_class=tokenizer,
)

max_steps is given, it will override any value given in num_train_epochs


In [14]:
trainer.train()

NOTE: Using for loop to run decoder layers


Epoch,Training Loss,Validation Loss
0,5.496500,5.361655


TrainOutput(global_step=4000, training_loss=5.915705322265625, metrics={'train_runtime': 1125.3595, 'train_samples_per_second': 227.483, 'train_steps_per_second': 3.554, 'total_flos': 8.3505639849984e+16, 'train_loss': 5.915705322265625, 'epoch': 0.1421110597932284})

In [15]:
import torch_xla.debug.metrics as met
print(met.short_metrics_report())
met.clear_all()

Counter: CachedCompile
  Value: 8139
Metric: CompileTime
  TotalSamples: 69
  Accumulator: 01m11s120ms701.270us
  ValueRate: 066ms564.830us / second
  Rate: 0.0636107 / second
  Percentiles: 1%=016ms343.669us; 5%=016ms498.910us; 10%=017ms598.440us; 20%=018ms714.550us; 50%=019ms575.050us; 80%=020ms217.039us; 90%=058ms739.589us; 95%=06s942ms973.927us; 99%=39s543ms724.966us
Metric: ExecuteReplicatedTime
  TotalSamples: 8208
  Accumulator: 09m30s118ms628.649us
  ValueRate: 473ms710.038us / second
  Rate: 8.12761 / second
  Percentiles: 1%=007ms678.049us; 5%=010ms987.030us; 10%=018ms994.459us; 20%=018ms259.150us; 50%=028ms071.749us; 80%=106ms075.548us; 90%=107ms778.038us; 95%=107ms076.217us; 99%=108ms515.718us
Metric: TransferToDeviceTime
  TotalSamples: 20541
  Accumulator: 02s691ms241.802us
  ValueRate: 002ms588.671us / second
  Rate: 19.3117 / second
  Percentiles: 1%=055.190us; 5%=059.290us; 10%=061.810us; 20%=064.630us; 50%=075.140us; 80%=098.970us; 90%=105.770us; 95%=122.570us; 99%=16

In [16]:
del model
import gc
gc.collect()

0

## Train again, this time using scan

In [17]:
from transformers import LlamaConfig, LlamaForCausalLM

reset_seed()

# Define model configuration
config = LlamaConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=1024,  # Model size
    num_hidden_layers=28,  # Number of transformer layers
    num_attention_heads=8,  # Number of attention heads
    intermediate_size=2048,  # Size of the hidden feedforward layer
    max_position_embeddings=128,  # Max tokens in a sequence
    use_cache=False,
    unroll_decoders=False,
)

# Instantiate the model
model = LlamaForCausalLM(config)

In [18]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"].shuffle(seed=42),  # type:ignore
    eval_dataset=lm_datasets["validation"].shuffle(seed=42),  # type:ignore
    processing_class=tokenizer,
)

trainer.train()

max_steps is given, it will override any value given in num_train_epochs


NOTE: Using scan_layers to speed up compilation


Epoch,Training Loss,Validation Loss
0,5.496400,5.361534


TrainOutput(global_step=4000, training_loss=5.9156449584960935, metrics={'train_runtime': 8777.0858, 'train_samples_per_second': 29.167, 'train_steps_per_second': 0.456, 'total_flos': 8.3505639849984e+16, 'train_loss': 5.9156449584960935, 'epoch': 0.1421110597932284})

We can see that the for loop and scan model train to the same loss over time.

In [19]:
import torch_xla.debug.metrics as met
print(met.short_metrics_report())
met.clear_all()

Counter: CachedCompile
  Value: 8202
Metric: CompileTime
  TotalSamples: 6
  Accumulator: 34s537ms668.277us
  ValueRate: 004ms825.021us / second
  Rate: 0.000684329 / second
  Percentiles: 1%=03s587ms571.823us; 5%=03s587ms571.823us; 10%=03s587ms571.823us; 20%=03s088ms129.233us; 50%=06s646ms065.223us; 80%=08s915ms679.426us; 90%=09s820ms070.436us; 95%=09s820ms070.436us; 99%=09s820ms070.436us
Metric: ExecuteReplicatedTime
  TotalSamples: 8208
  Accumulator: 12m46s869ms342.175us
  ValueRate: 083ms579.890us / second
  Rate: 1.07778 / second
  Percentiles: 1%=021ms105.969us; 5%=022ms332.640us; 10%=023ms903.870us; 20%=024ms949.790us; 50%=027ms231.439us; 80%=149ms236.057us; 90%=150ms545.617us; 95%=150ms300.906us; 99%=151ms025.827us
Metric: TransferToDeviceTime
  TotalSamples: 209355
  Accumulator: 26s776ms849.127us
  ValueRate: 004ms228.496us / second
  Rate: 38.2862 / second
  Percentiles: 1%=073.370us; 5%=081.360us; 10%=084.510us; 20%=088.070us; 50%=096.530us; 80%=119.130us; 90%=155.340us; 9

## Verify the numerical correctness of `scan_layers`

Under the same weights, and the same input tokens, both the for loop based
implementation and `scan_layers` based implementation should produce the same
output tokens.

In [20]:
import torch_xla
input_ids = torch.tensor(tokenized_datasets["train"][3]["input_ids"]).unsqueeze(0).type(torch.LongTensor) # type:ignore
attention_mask = torch.tensor(tokenized_datasets["train"][3]["attention_mask"]).unsqueeze(0) # type:ignore
input_ids = input_ids.to(torch_xla.device())
attention_mask = attention_mask.to(torch_xla.device())
torch_xla.sync()

In [21]:
input_ids

tensor([[128000,   5476,     73,  56761,    912,  86262,     88,   4298,    220,
             18,    551,    366,   3200,     29,  66416,    320,  11002,    551,
          50534,     99,  75267,  16144, 115687,  33710, 123283, 104612,     18,
           1174,  13318,    662,  86262,     88,   4298,    315,    279,  71735,
            220,     18,    883,   1174,  17037,  14183,    311,    439,  86262,
             88,   4298,  66416,  14767,   4994,   6457,   1174,    374,    264,
          39747,   3560,    571,     12,     31,   5737,   2835,   1847,   8040,
            555,  80949,    323,   7972,   5168,   1854,    369,    279,  32365,
          42585,    662,  45894,    304,   6186,    220,    679,     16,    304,
           6457,   1174,    433,    374,    279,   4948,   1847,    304,    279,
          86262,     88,   4298,   4101,    662,  21445,    287,    279,   1890,
          37608,    315,  39747,    323,   1972,    571,     12,     31,    892,
          27120,    439,   1

In [22]:
model.model.unroll_decoders = False
model.model.logged_messages = set()
logits = model.forward(input_ids, attention_mask).logits  # type:ignore
logits.shape, logits

NOTE: Using scan_layers to speed up compilation


(torch.Size([1, 128, 128000]),
 tensor([[[-5.1699, -3.6749, -4.5099,  ..., -4.1783, -4.7814, -4.8963],
          [-3.8756, -3.2109, -3.7001,  ..., -3.9365, -3.8906, -4.1175],
          [-3.8278, -3.3501, -3.6548,  ..., -3.9579, -3.8159, -4.0969],
          ...,
          [-5.7814, -5.6866, -6.0840,  ..., -5.2086, -6.3461, -5.3524],
          [-5.6320, -5.2927, -6.1013,  ..., -5.0950, -6.1226, -5.2884],
          [-6.3101, -6.1876, -6.6119,  ..., -5.9928, -6.0297, -5.7701]]],
        device='xla:0', grad_fn=<UnsafeViewBackward0>))

In [23]:
def pick_token(logits):
  return torch.argmax(logits, dim=-1)

In [24]:
tokens = pick_token(logits)
tokens

tensor([[ 284,  571,  301,  574, 1174, 1174,  320,  320,   17,  320,  220, 3200,
           29,  366, 1174,  220,  551,  366,  366,  366,  366,  366,  366,  366,
          883,  883,  883,  220,  366,  220,  571,  366,  883,  279,  220,  571,
           17,  571,  374,  574, 3967,  311,  279,  264,  571, 1174, 1174, 1174,
          662,  279,  662,  323,  264,  220,  571,  304,   12,   31,  709, 1174,
         1847,  662,  555,  279,  662,  279,  662,   13,  662,  279,  220,  220,
          662,  578,  304,  220,  220,  679,   15, 1174,  279, 1174,  279,  574,
          264, 1176,  571,  304,  279,  220,  315,  315,  662,  315,  720,  220,
          279, 1847, 1060, 1174,  279,  571,  279,  571,   12,   31,  892, 1174,
         1174, 1664, 1176, 1174,  323, 1176,  374,  389,  311,  279, 1847,  892,
          304,  279,  279, 1847,  366,  330,  330,  662]], device='xla:0')

In [25]:
tokenizer.decode(tokens[0].detach().cpu().numpy().tolist())

' = @el was,, ( (2 ( unk> <,  : < < < < < < < ) ) )  <  @ < ) the  @2 @ is was known to the a @,,,. the. and a  @ in-@ up, game. by the. the... the  . The in  2010, the, the was a first @ in the  of of. of \n  the game year, the @ the @-@ time,, well first, and first is on to the game time in the the game < " ".'

In [26]:
model.model.unroll_decoders = True
model.model.logged_messages = set()
for_loop_logits = model.forward(input_ids, attention_mask).logits  # type:ignore
for_loop_logits.shape, for_loop_logits

NOTE: Using for loop to run decoder layers


(torch.Size([1, 128, 128000]),
 tensor([[[-5.1699, -3.6749, -4.5099,  ..., -4.1783, -4.7814, -4.8963],
          [-3.8756, -3.2109, -3.7001,  ..., -3.9365, -3.8906, -4.1175],
          [-3.8278, -3.3501, -3.6548,  ..., -3.9579, -3.8159, -4.0969],
          ...,
          [-5.7814, -5.6866, -6.0840,  ..., -5.2086, -6.3461, -5.3524],
          [-5.6320, -5.2927, -6.1013,  ..., -5.0950, -6.1226, -5.2884],
          [-6.3101, -6.1876, -6.6119,  ..., -5.9928, -6.0297, -5.7701]]],
        device='xla:0', grad_fn=<UnsafeViewBackward0>))

In [27]:
for_loop_tokens = pick_token(for_loop_logits)
for_loop_tokens

tensor([[ 284,  571,  301,  574, 1174, 1174,  320,  320,   17,  320,  220, 3200,
           29,  366, 1174,  220,  551,  366,  366,  366,  366,  366,  366,  366,
          883,  883,  883,  220,  366,  220,  571,  366,  883,  279,  220,  571,
           17,  571,  374,  574, 3967,  311,  279,  264,  571, 1174, 1174, 1174,
          662,  279,  662,  323,  264,  220,  571,  304,   12,   31,  709, 1174,
         1847,  662,  555,  279,  662,  279,  662,   13,  662,  279,  220,  220,
          662,  578,  304,  220,  220,  679,   15, 1174,  279, 1174,  279,  574,
          264, 1176,  571,  304,  279,  220,  315,  315,  662,  315,  720,  220,
          279, 1847, 1060, 1174,  279,  571,  279,  571,   12,   31,  892, 1174,
         1174, 1664, 1176, 1174,  323, 1176,  374,  389,  311,  279, 1847,  892,
          304,  279,  279, 1847,  366,  330,  330,  662]], device='xla:0')

In [28]:
tokenizer.decode(for_loop_tokens[0].detach().cpu().numpy().tolist())

' = @el was,, ( (2 ( unk> <,  : < < < < < < < ) ) )  <  @ < ) the  @2 @ is was known to the a @,,,. the. and a  @ in-@ up, game. by the. the... the  . The in  2010, the, the was a first @ in the  of of. of \n  the game year, the @ the @-@ time,, well first, and first is on to the game time in the the game < " ".'

In [29]:
torch.allclose(logits, for_loop_logits, atol=1e-5, rtol=1e-5)

True

# Test the gradients of scan

After I run both scan and for loop versions of the model on the same input, their
gradients should also be similar.

In [30]:
torch_xla.sync()

input_ids.requires_grad_(False)
attention_mask.requires_grad_(False)
labels = input_ids[:, :].clone().contiguous()

# Run for loop model and collect the gradients
reset_seed()
for_loop_grads = []
model.zero_grad()
model.model.zero_grad()
torch_xla.sync()
model.model.unroll_decoders = True
model.model.logged_messages = set()
with torch.enable_grad():
  model(input_ids, attention_mask, labels=labels).loss.backward()  # type: ignore
torch_xla.sync()
for (name, param) in model.named_parameters():
  assert param.grad is not None
  for_loop_grads.append((name, param.grad.clone().detach()))

# Run scan model and collect the gradients
reset_seed()
scan_grads = []
model.zero_grad()
model.model.zero_grad()
torch_xla.sync()
model.model.unroll_decoders = False
model.model.logged_messages = set()
with torch.enable_grad():
  model(input_ids, attention_mask, labels=labels).loss.backward()  # type: ignore
torch_xla.sync()
for (name, param) in model.named_parameters():
  assert param.grad is not None
  scan_grads.append((name, param.grad.clone().detach()))

NOTE: Using for loop to run decoder layers
NOTE: Using scan_layers to speed up compilation


In [31]:
# Compare the gradients
assert len(for_loop_grads) == len(scan_grads)
assert len(for_loop_grads) > 0

In [32]:
for ((for_loop_name, for_loop_grad), (scan_name, scan_grad)) in zip(for_loop_grads, scan_grads):
  assert for_loop_name == scan_name
  assert torch.allclose(for_loop_grad, scan_grad, atol=1e-5, rtol=1e-5), f"{for_loop_name} mismatch by: {torch.max(torch.abs(for_loop_grad - scan_grad))}"
  print(f"Pass: {for_loop_name}")

Pass: model.embed_tokens.weight
Pass: model.layers.0.self_attn.q_proj.weight
Pass: model.layers.0.self_attn.k_proj.weight
Pass: model.layers.0.self_attn.v_proj.weight
Pass: model.layers.0.self_attn.o_proj.weight
Pass: model.layers.0.mlp.gate_proj.weight
Pass: model.layers.0.mlp.up_proj.weight
Pass: model.layers.0.mlp.down_proj.weight
Pass: model.layers.0.input_layernorm.weight
Pass: model.layers.0.post_attention_layernorm.weight
Pass: model.layers.1.self_attn.q_proj.weight
Pass: model.layers.1.self_attn.k_proj.weight
Pass: model.layers.1.self_attn.v_proj.weight
Pass: model.layers.1.self_attn.o_proj.weight
Pass: model.layers.1.mlp.gate_proj.weight
Pass: model.layers.1.mlp.up_proj.weight
Pass: model.layers.1.mlp.down_proj.weight
Pass: model.layers.1.input_layernorm.weight
Pass: model.layers.1.post_attention_layernorm.weight
Pass: model.layers.2.self_attn.q_proj.weight
Pass: model.layers.2.self_attn.k_proj.weight
Pass: model.layers.2.self_attn.v_proj.weight
Pass: model.layers.2.self_attn.

In [33]:
for_loop_grads[3], torch.max(for_loop_grads[3][1]), torch.min(for_loop_grads[3][1])

(('model.layers.0.self_attn.v_proj.weight',
  tensor([[ 6.7186e-05,  1.6983e-04,  4.7841e-04,  ..., -1.6211e-04,
            3.0965e-04,  1.3230e-03],
          [ 3.8496e-03, -2.2651e-03,  9.5748e-05,  ...,  6.6169e-04,
            2.5951e-03, -4.4781e-03],
          [ 1.4842e-03, -3.7740e-04, -1.4062e-03,  ...,  9.7732e-04,
            2.5738e-03, -2.3177e-03],
          ...,
          [ 3.9044e-04,  8.9442e-04, -3.1140e-04,  ..., -2.0575e-03,
           -2.3838e-03,  2.9901e-04],
          [-1.0933e-04,  1.2664e-03, -1.2857e-03,  ..., -1.6495e-03,
           -8.3783e-04,  2.4912e-03],
          [ 6.9341e-04, -1.1699e-03,  5.8735e-04,  ..., -6.4733e-04,
            2.2107e-03,  3.0538e-05]], device='xla:0')),
 tensor(0.0275, device='xla:0'),
 tensor(-0.0279, device='xla:0'))

In [34]:
scan_grads[3], torch.max(scan_grads[3][1]), torch.min(scan_grads[3][1])

(('model.layers.0.self_attn.v_proj.weight',
  tensor([[ 6.7186e-05,  1.6983e-04,  4.7841e-04,  ..., -1.6211e-04,
            3.0965e-04,  1.3230e-03],
          [ 3.8496e-03, -2.2651e-03,  9.5748e-05,  ...,  6.6169e-04,
            2.5951e-03, -4.4781e-03],
          [ 1.4842e-03, -3.7740e-04, -1.4062e-03,  ...,  9.7732e-04,
            2.5738e-03, -2.3177e-03],
          ...,
          [ 3.9044e-04,  8.9442e-04, -3.1140e-04,  ..., -2.0575e-03,
           -2.3838e-03,  2.9901e-04],
          [-1.0933e-04,  1.2664e-03, -1.2857e-03,  ..., -1.6495e-03,
           -8.3783e-04,  2.4912e-03],
          [ 6.9341e-04, -1.1699e-03,  5.8735e-04,  ..., -6.4733e-04,
            2.2107e-03,  3.0538e-05]], device='xla:0')),
 tensor(0.0275, device='xla:0'),
 tensor(-0.0279, device='xla:0'))